<a href="https://colab.research.google.com/github/Ransinghsatyajitray/Simple_Chatbot/blob/main/Chatbot_Llama2_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. transformers -> for connecting with Huggingface
2. datasets -> for getting a wide array of pre-processed datasets for machine learning tasks without the hassle of manual collection and cleaning.
3. loralib ->  LoRA is a technique that allows for efficient fine-tuning of large language models, reducing the number of trainable parameters and memory usage.
4. sentencepiece -> SentencePiece efficiently tokenizes text into subwords, handling out-of-vocabulary words and improving language model performance.
5. bitsandbytes -> reduces the memory footprint of large language models, enabling them to run on consumer-grade hardware without sacrificing performance.
6. accelerate ->  simplifies distributed training and optimization for PyTorch models, making it easier to leverage multiple GPUs or TPUs.
7. xformers ->  provides tools for building and training Transformer-based models for natural language processing (NLP) tasks.
8. einops ->  simplifies tensor manipulations for deep learning with its intuitive and expressive notation, improving code readability and maintainability.
9. langchain -> simplifies building LLM-powered applications by providing a standardized interface for various chains, agents, and data sources.

In [3]:
!pip -q install datasets loralib sentencepiece bitsandbytes accelerate xformers einops langchain
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [14]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.7 MB/s eta 0:00:00


In [4]:
!nvidia-smi

Fri Jul  5 07:02:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Connecting to Huggingfacehub to use the Llama2 model

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

# Making the tokenizer and model for the pretained Llama-2-7b-chat-hf model
torch was used to reduce the memory (using the quantization internally, see torch_dtype = torch.float16)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

# Pipeline is used for knitting the tokenizer , model. Other parameters are kept as default.

In [8]:
# Use a pipeline for later
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [9]:
!nvidia-smi

Fri Jul  5 07:07:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0              28W /  70W |  12957MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Prompting in a chatmodel particularly with llama2 usually take 3 features/roles ->
1. System Prompt -> It is like guiding the overall behaivior of the model
2. User -> The query/input instruction which the user gives
3. AI -> This is how the model would respond and the output depends on the how System Prompt is defined and what input the user gives

B_SYS, E_SYS -> They encapsulate the system message, which provides the model with context, guidelines, or rules about how it should behave or respond.
B_INST, E_INST -> They are used to clearly delineate the instruction or prompt given to the LLM, separating it from the system message or the model's response.




In [10]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

# Utility functions
def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")


# Function for returning the final output with Context Manager specifying how the code would run(resource it would be needing)
def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

# Parsing the text to view the output
def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


# Checking the templates

In [11]:
instruction = "What is the temperature in Delhi?"

get_prompt(instruction)

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\nWhat is the temperature in Delhi?[/INST]"

In [12]:
instruction = "Summarize the following text for me {text}"

system_prompt = "You are an expert and summarization and reducing the number of words used"

get_prompt(instruction, system_prompt)

'[INST]<<SYS>>\nYou are an expert and summarization and reducing the number of words used\n<</SYS>>\n\nSummarize the following text for me {text}[/INST]'

# Using the HuggingFacePipeline and setting the temperature to 0 to avoid any randomness in the output.

In [15]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain

In [16]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


# Making the chain using the system prompt and instruction and the llm model


In [17]:
system_prompt = "You are an advanced assistant that excels at translation. "
instruction = "Convert the following text from English to Hindi:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

[INST]<<SYS>>
You are an advanced assistant that excels at translation. 
<</SYS>>

Convert the following text from English to Hindi:

 {text}[/INST]


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


# Testing the model with a translation task

In [20]:
text = "how are you today?"
output = llm_chain.run(text)

parse_text(output)

[INST]<<SYS>> You are an advanced assistant that excels at translation.  <</SYS>>  Convert the
following text from English to Hindi:   how are you today?[/INST]  Sure, I'd be happy to help!
Here's the translation of "how are you today?" from English to Hindi:  हाँ, आज कैसे हैं आप? (Haan,
aaj kase hain aap?)




# Simple ChatBot using the Langchain. It stores info in buffer memory


In [35]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate

In [36]:
instruction = "Chat History:\n\n{chat_history} \n\nUser: {user_input}"
system_prompt = "You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context"

template = get_prompt(instruction, system_prompt)
print(template)

[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

{chat_history} 

User: {user_input}[/INST]


In [37]:

prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [38]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

# Checking the response from the model

In [39]:
llm_chain.predict(user_input="Hi, my name is Ransingh")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

User: Hi, my name is Ransingh[/INST]

> Finished chain.


'[INST]<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<</SYS>>\n\nChat History:\n\n \n\nUser: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today?'

In [40]:
llm_chain.predict(user_input="Can you tell me about yourself.")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset




> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Ransingh
AI: [INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

User: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today? 

User: Can you tell me about yourself.[/INST]

> Finished chain.


"[INST]<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<</SYS>>\n\nChat History:\n\nHuman: Hi, my name is Ransingh\nAI: [INST]<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<</SYS>>\n\nChat History:\n\n \n\nUser: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today? \n\nUser: Can you tell me about yourself.[/INST]  Of course! I'm here to help you with any questions or tasks you may have. Is there something specific you'd like to know or discuss?"

In [42]:
llm_chain.predict(user_input="Today is Friday. What number day of the week is that?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Ransingh
AI: [INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

User: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today?
Human: Can you tell me about yourself.
AI: [INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Ransingh
AI: [INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

User: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today? 

User: Can y

"[INST]<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<</SYS>>\n\nChat History:\n\nHuman: Hi, my name is Ransingh\nAI: [INST]<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<</SYS>>\n\nChat History:\n\n \n\nUser: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today?\nHuman: Can you tell me about yourself.\nAI: [INST]<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<</SYS>>\n\nChat History:\n\nHuman: Hi, my name is Ransingh\nAI: [INST]<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<</SYS>>\n\nChat History:\n\n \n\nUser: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today? \n\nUser: Can you tell me about yourself

In [43]:
llm_chain.predict(user_input="what is the day today?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Ransingh
AI: [INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

User: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today?
Human: Can you tell me about yourself.
AI: [INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Ransingh
AI: [INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

User: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today? 

User: Can y

"[INST]<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<</SYS>>\n\nChat History:\n\nHuman: Hi, my name is Ransingh\nAI: [INST]<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<</SYS>>\n\nChat History:\n\n \n\nUser: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today?\nHuman: Can you tell me about yourself.\nAI: [INST]<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<</SYS>>\n\nChat History:\n\nHuman: Hi, my name is Ransingh\nAI: [INST]<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<</SYS>>\n\nChat History:\n\n \n\nUser: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today? \n\nUser: Can you tell me about yourself

In [45]:
llm_chain.predict(user_input="Where is Bhubaneswar")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Ransingh
AI: [INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

User: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today?
Human: Can you tell me about yourself.
AI: [INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Ransingh
AI: [INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

User: Hi, my name is Ransingh[/INST]  Hello Ransingh! How may I assist you today? 

User: Can y

OutOfMemoryError: CUDA out of memory. Tried to allocate 52.00 MiB. GPU 